# Project on Calculating the Head Count

In [10]:
#Importing libraries
import numpy as np 
import cv2
import imutils
import datetime

In [15]:
#This function will decide whether the person is going IN or going OUT
#It will return 2 elements, first the element itself and second its occurrances
def find_majority_element(lst):
    Map={}          #declare map for storing values of lst
    maximum=('',0)   #(occurring element, occurrences)
    for i in lst:
        if i in Map:
            Map[i]+=1      #Increment the value of element in Map if it is encountered more than once
        else:
            Map[i]=1        #When the element is encountered at first
        #Keep track of maximum occurring element    
        if Map[i]>maximum[1]:
            maximum=(i,Map[i])
    return maximum        
            

In [20]:
count1=0 #When the person enters in the room (IN) (i.e;Total persons IN)
count2=0 #When the element goes out of the room (OUT) (i.e; Total perons OUT)

avgFrame=None   #This variable stores the first frame in grascale of the video

xvalues=list()  #TO store values of X-coordinates of a person moving in Video
                #1.If the person is moving from left to right(IN) the xvalues would be like (100,120,150,200,240,350,400)
                #2.If the person is moving from right to left(OUT) the xvalues would be like (400,360,340,330,300,250,200,150)

    
motion=list()       #This can only stores the value 0 or 1 based on  xvalues(as defined above)
                    #Later in the code we'll see how values in this tuple is filled
                
                

#Creating the object of VideoCapture to read the different frames from the Video
#We'll be passing the path of Video file in itsparameter
#We can also Pass 0 as its parameter in this case Builtin camera of the system will get opened and will capture the first frame
video=cv2.VideoCapture("myvideo.mp4")

#Since the Video is combination of large number of Frames
#Now we'll be iterating over each frames
while True:
    
    #This will add the window and start reading the frame. It will  return 2 things.
    #1. check->bool type True if object(video) is able to read the frame
    #2. frame->numpy array that represents the first image that video captures
    check,frame=video.read()
    
    flag=True   #To indicate whether the person is in the frame (True menas no person is in the frame)

    #When our Video is over the check will return false
    if(check==False):
        break
    
    #Image Preprocessing
    #1. First we'll resize our frame because larger image reqiures more processing time
    #2. Second We'll Convert the Coloured frame to gray Scale image and store it in variable gray
    #3. Then further we will blur that grau image so as to reduce the noise 
    frame=imutils.resize(frame,width=500)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray=cv2.GaussianBlur(gray,(21,21),0)
    
    
    #The following if statement will run only once and will initialize the first frame
    #This first frame is stored in variable->avgFrame
    if avgFrame is None:
        print("Model Started")
        avgFrame=gray.copy().astype("float") #storing a copy of grayscale image in avgFrame
        continue     #due to this 'continue' from here first iteration is skipped(Remember First frame is already stored)
        
        
    
    #Now, in Second itertation we accumulate the weighted average between the current frame and first frame
    #Then we compute the difference between the current(Running) frame and First Frame
    cv2.accumulateWeighted(gray,avgFrame,0.5)
    frameDelta=cv2.absdiff(gray,cv2.convertScaleAbs(avgFrame))
    
    
    #Now we'll be forming the threshold image with the help of frameDelta calculated above.
    #This Threshold image is formed so that we can find the "countours" in the Frame.
    thresh=cv2.threshold(frameDelta,5,255,cv2.THRESH_BINARY)[1] #cv2.THRESH_BINARY is a method of thresholding
    thresh=cv2.dilate(thresh,None,iterations=2)     #We dilate the threshold image to fill the holes
    
    
    #Now we'll find countours in this threshold image(A contour refers to the outline of an object)
    #To find contours in an image, we need the OpenCV "cv2.findContours" function
    #It Accepts 3 parameters-
      #1.Copy of threshold image(Because this function is distructive in nature so we pass the copy)
      #2.cv2.RETR_EXTERNAL tells OpenCV to compute the hierarchy (relationship) between contours  
      #3.We tell OpenCV to compress the contours to save space using cv2.CV_CHAIN_APPROX_SIMPLE.
    (cnts,_)=cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    

        
        
    #Now iterating over countours
    for c in cnts:
        #If the countour is too small the ignore it
        if cv2.contourArea(c)<5000:
            continue
        #make the bounding rectangle if see the person(countours)   
        #After that, store the x coordinate of the person moving in the frame
        (x,y,w,h)=cv2.boundingRect(c)
        xvalues.append(x)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        flag=False #It means that person is still in the frame
                   #Count is only made when the person goes out of the frame this can be seen in below code.
        
    
    #Calculating the length of list consisting of  X-coordinates 
    no_x = len(xvalues)
    
    #With the help of below if statement we'll insert values on our motion list
    if (no_x > 2):
        #If the difference between last two values is greater than 0, it means the person is moving from left to right(IN).
        #for example if xvalues->(100,120,150,200,240,350,400) then 400-350 >0 so persin is moving IN and we append 1 to motion.
        difference = xvalues[no_x - 1] - xvalues[no_x - 2]
        if(difference > 0):
            motion.append(1)
        else:
            motion.append(0)
        #Similarly,If the difference between last two values is less than 0, it means the person is moving from right to left(OUT).
        #for example if xvalues->(400,360,340,330,300,250,200,150) then,150-200<0 so persin is moving OUT and we append 0 to motion   
            
            
    #print("xlaues-->",xvalues)
    #print("motion-->",motion)
    
    
    #The below if condition will run if the person is out of the frame since flag is True
    if flag is True:
        if (no_x > 5):
            #This function will return the majority element in the list.
            #On the basis of this we determine whether the person is going IN or OUT.
            val, times = find_majority_element(motion) #for definition see above
            if val == 1 and times >= 15:  #Means person is coming IN
                count1 += 1
            else:
                count2 += 1               #Means Person is going OUT
        
        #Reset the xvalues and motion to NULL
        xvalues = list()
        motion = list()
        
    #For drawing the straight line in the frame on appropriate place
    cv2.line(frame,(260,0),(260,480),(0,0,255),2)
    cv2.line(frame,(320,0),(320,480),(0,255,0),2)
    
    #Placing the text on the frame
    cv2.putText(frame,"IN: {}".format(count1),(10,20),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
    cv2.putText(frame,"OUT: {}".format(count2),(10,40),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
    
    #This will display the current date and time(implemented with the help of datetime library imported above).
    cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
                    (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
    
    #This statement will capture the frame and show it
    #Since the frame are being read in a whine loop so it will appear like a video
    cv2.imshow("Frame",frame)
    cv2.imshow("Gray",gray)
    cv2.imshow('FrameDelta',frameDelta)
    
    
    #if we'll press key 'q' then it will break from while loop
    key=cv2.waitKey(10) & 0xFF
    if(key==ord('q')):
        break
        
video.release()   #This will release the video file in few milliseconds
cv2.destroyAllWindows() #This will Close any open window

                          #####################################
                                         #END#
                         ######################################

Model Started
